In [1]:
import torch
from torchessian import hessian_matmul, lanczos
from itertools import product

In [2]:
n = 2
model = torch.nn.Linear(n, 1)
# torch.manual_seed(2019)
v = torch.rand(sum(p.data.numel() for p in model.parameters()))
x = torch.rand(10, n)
y = x.sum(1)
def loss_function(y_hat, y):
    return ((y_hat.view(-1) - y.view(-1)) * (y_hat.view(-1) - y.view(-1))).sum() / y.numel()
batch = x, y
m = 3
# Exact Hessian

h11 = 2 * (x[:, 0] * x[:, 0]).sum() / x.size(0)
h21 = h12 = 2 * (x[:, 0] * x[:, 1]).sum() / x.size(0)
h31 = h13 = 2 * x[:, 0].sum() / x.size(0)

h22 = 2 * (x[:, 1] * x[:, 1]).sum() / x.size(0)
h32 = h23 = 2 * x[:, 1].sum() / x.size(0)

h33 = 2

H = torch.zeros(3, 3)

for i, j in product(range(3), range(3)):
    exec("H[%d, %d] = h%d%d" % (i, j, i+1, j+1))
    
print(H)

tensor([[0.6388, 0.5488, 0.8922],
        [0.5488, 0.8968, 1.1895],
        [0.8922, 1.1895, 2.0000]])


In [3]:
T, V = lanczos(model, loss_function, batch, m)

In [4]:
v = torch.eig(T)[0]
v.sort()
v

tensor([[3.1897, 0.0000],
        [0.1338, 0.0000],
        [0.2123, 0.0000]])

In [5]:
v = torch.eig(H)[0]
v.sort()
v

tensor([[3.1896, 0.0000],
        [0.2123, 0.0000],
        [0.1337, 0.0000]])

In [6]:
for i in range(m - 1):
    print("{:.3f}".format(V[i, :].dot(V[m - 1, :])))

-0.000
0.000


In [7]:
w, v = torch.eig(T, eigenvectors=True)

In [9]:
v

tensor([[ 0.9565,  0.2230,  0.1883],
        [ 0.2918, -0.7393, -0.6068],
        [ 0.0039,  0.6353, -0.7722]])

In [10]:
v.transpose(0, 1).matmul(T.matmul(v))

tensor([[ 3.1897e+00, -1.7399e-07, -1.1940e-07],
        [-2.6077e-07,  1.3379e-01,  1.5772e-08],
        [-2.0082e-07,  2.5792e-08,  2.1228e-01]])